In [2]:
%pip install -U -q google-generativeai

DEPRECATION: pytorch-lightning 1.5.6 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import textwrap
import google.generativeai as genai
from IPython.display import Markdown

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import smtplib as smtp
from getpass import getpass

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [2]:
api_key = ""

In [3]:
genai.configure(api_key=api_key)

In [4]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b



def send_email(destination_email: str, message_text: str):
    """Send message to user_email with message_text"""

    email = ''
    password = ''
    dest_email = destination_email
    subject = 'Test'
    email_text = message_text

    message = 'From: {}\nTo: {}\nSubject: {}\n\n{}'.format(email,
                                                        dest_email, 
                                                        subject, 
                                                        email_text)

    server = smtp.SMTP_SSL('smtp.yandex.com')
    server.set_debuglevel(1)
    server.ehlo(email)
    server.login(email, password)
    server.auth_plain()
    server.sendmail(email, dest_email, message)
    server.quit()

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[multiply, send_email])

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x10de0d7c0>,
    system_instruction=None,
)

In [9]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [6]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

"That's a total of 2508 mittens. \n"

In [ ]:
response = chat.send_message('Send a message to email about meeting on June 13. And tell him to call my number back. Be kind')
response.text

In [14]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'Send a message to alanrbtx@gmail.com about the MTS AI meeting on June 13. Your name is Newton'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'send_email', 'args': {'message_text': 'Hi, this is Newton. Just a reminder about the MTS AI meeting on June 13.', 'destination_email': 'alanrbtx@gmail.com'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'send_email', 'response': {'result': None}}}
--------------------------------------------------------------------------------
model -> {'text': "OK. I've sent an email to alanrbtx@gmail.com. \n"}
--------------------------------------------------------------------------------
user -> {'text': 'Send a message to alanrbtx@gmail.com about the MTS AI meeting on June 13. And tell him to call my number back. Be kind'}
-------------------------------------------------------------------------

In [15]:
response = chat.send_message('How are you?')
response.text

"As an AI, I don't have feelings like humans do.  But I'm here to assist you with any questions or tasks you may have! 😊  How can I help you today? \n"